In [3]:
import mysql.connector
from mysql.connector import Error

# Function to create a connection to MySQL
def create_connection():
    connection = None
    try:
        connection = mysql.connector.connect(
            host='localhost',
            database='community_hub',
            user='root',
            password='Sagarite@710'
        )
        if connection.is_connected():
            print("Connected to MySQL database")
            return connection
    except Error as e:
        print(f"Error: {e}")
    return connection

def execute_query(connection, query):
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        connection.commit()
        return cursor
    except Exception as e:
        print(f"Error executing query: {e}")
        return None

def fetch_data(connection, query):
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        data = cursor.fetchall()
        column_names = [desc[0] for desc in cursor.description]
        return column_names, data
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None, None


In [4]:
# Create influencers table
create_influencers_table_query = """
CREATE TABLE influencers (
    influencer_id INT PRIMARY KEY,
    first_name VARCHAR(255),
    last_name VARCHAR(255),
    dob DATE,
    gender CHAR(1) CHECK(gender IN ('M', 'F')),
    email VARCHAR(255),
    phone_number VARCHAR(15),
    date_joined DATE,
    category VARCHAR(255),
    assets DECIMAL(10, 2),
    liabilities DECIMAL(10, 2),
    bus_vent_gp DECIMAL(10, 2),
    expenses DECIMAL(10, 2)
);
"""

# Create Course table
create_course_table_query = """
CREATE TABLE Course (
    influencer_id INT,
    Course VARCHAR(255),
    College_name VARCHAR(255),
    End_date DATE,
    Enroll_date DATE,
    PRIMARY KEY (influencer_id, Course),
    FOREIGN KEY (influencer_id) REFERENCES influencers (influencer_id) ON DELETE CASCADE
);
"""

# Create Instagram table
create_instagram_table_query = """
CREATE TABLE Instagram (
    account_name VARCHAR(255) PRIMARY KEY,
    influencer_id INT,
    ad_revenue DECIMAL(10, 2),
    joining_date DATE,
    followers INT,
    engagement_rate DECIMAL(5, 2),
    audience_demographics JSON,
    FOREIGN KEY (influencer_id) REFERENCES influencers (influencer_id) ON DELETE CASCADE
);
"""

# Create posts table
create_posts_table_query = """
CREATE TABLE posts (
    post_id INT PRIMARY KEY,
    account_name VARCHAR(255),
    post_date DATE,
    likes INT,
    comments INT,
    post_type ENUM('Image', 'Video', 'Story'),
    FOREIGN KEY (account_name) REFERENCES Instagram (account_name) ON DELETE CASCADE
);
"""

# Create collaborations table
create_collaborations_table_query = """
CREATE TABLE collaborations (
    collaboration_id INT PRIMARY KEY,
    brand_name VARCHAR(255),
    influencer_id INT,
    collaboration_earnings DECIMAL(10, 2),
    collaboration_type ENUM('Product Placement', 'Sponsored Event'),
    FOREIGN KEY (influencer_id) REFERENCES influencers (influencer_id) ON DELETE CASCADE
);
"""

# Create community_engagement table
create_community_engagement_table_query = """
CREATE TABLE community_engagement (
    engagement_id INT PRIMARY KEY,
    influencer_id INT,
    event_name VARCHAR(255),
    event_date DATE,
    participants INT,
    rating DECIMAL(5, 2),
    FOREIGN KEY (influencer_id) REFERENCES influencers (influencer_id) ON DELETE CASCADE
);
"""

# Create Analytics table
create_analytics_table_query = """
CREATE TABLE Analytics (
    influencer_id INT PRIMARY KEY,
    age INT,
    studying VARCHAR(255),
    avg_net_worth DECIMAL(10, 2),
    avg_reach INT,
    FOREIGN KEY (influencer_id) REFERENCES influencers (influencer_id) ON DELETE CASCADE
);
"""

In [5]:
# Create Trigger to update derived columns in Analytics table
create_trigger_query = """
-- Trigger to calculate age, studying, avg_net_worth, and avg_reach before insert
CREATE TRIGGER calculate_analytics_values
BEFORE INSERT ON Analytics
FOR EACH ROW
BEGIN
    SET NEW.age = YEAR(CURDATE()) - YEAR((SELECT dob FROM influencers WHERE influencer_id = NEW.influencer_id));
    
    SET NEW.studying = (
        SELECT 
            CASE 
                WHEN CURDATE() BETWEEN Enroll_date AND End_date THEN 'Yes'
                ELSE 'No'
            END
        FROM Course 
        WHERE influencer_id = NEW.influencer_id 
        ORDER BY End_date DESC 
        LIMIT 1
    );
    
    SET NEW.avg_net_worth = (
        SELECT 
            COALESCE(SUM(assets - liabilities + bus_vent_gp + ad_revenue + collaboration_earnings - expenses), 0)
        FROM 
            influencers
        LEFT JOIN Instagram ON influencers.influencer_id = Instagram.influencer_id
        LEFT JOIN collaborations ON influencers.influencer_id = collaborations.influencer_id
        WHERE influencers.influencer_id = NEW.influencer_id
    );
    
    SET NEW.avg_reach = (
        SELECT 
            COALESCE(SUM(participants + followers + engagement_rate + likes + comments), 0)
        FROM 
            Instagram
        LEFT JOIN posts ON Instagram.account_name = posts.account_name
        LEFT JOIN community_engagement ON Instagram.influencer_id = community_engagement.influencer_id
        WHERE Instagram.influencer_id = NEW.influencer_id
    );
END;
"""

In [6]:
# Connecting to MySQL
connection = create_connection()

# Executing queries to create tables
execute_query(connection, create_influencers_table_query)
execute_query(connection, create_course_table_query)
execute_query(connection, create_instagram_table_query)
execute_query(connection, create_posts_table_query)
execute_query(connection, create_collaborations_table_query)
execute_query(connection, create_community_engagement_table_query)
execute_query(connection, create_analytics_table_query)
execute_query(connection, create_trigger_query)

Connected to MySQL database
Error executing query: 1050 (42S01): Table 'influencers' already exists
Error executing query: 1050 (42S01): Table 'course' already exists
Error executing query: 1050 (42S01): Table 'instagram' already exists
Error executing query: 1050 (42S01): Table 'posts' already exists
Error executing query: 1050 (42S01): Table 'collaborations' already exists
Error executing query: 1050 (42S01): Table 'community_engagement' already exists
Error executing query: 1050 (42S01): Table 'analytics' already exists
Error executing query: 1359 (HY000): Trigger already exists


In [7]:
# Insert data into influencers, Course, Instagram, posts, collaborations, community_engagement tables
insert_influencers_query = """
INSERT INTO influencers 
    (influencer_id, first_name, last_name, dob, gender, email, phone_number, date_joined, category, assets, liabilities, bus_vent_gp, expenses)
VALUES 
    (1, 'Shrishti', 'Gupta', '1990-05-15', 'F', 'shrishti@example.com', '+1234567890', '2020-01-01', 'Fashion', 50000.00, 20000.00, 15000.00, 10000.00);
"""
execute_query(connection, insert_influencers_query)

insert_course_query = """
INSERT INTO Course 
    (influencer_id, Course, College_name, End_date, Enroll_date)
VALUES 
    (1, 'Fashion Design', 'Fashion Institute', '2022-12-31', '2020-01-15');
"""
execute_query(connection, insert_course_query)

insert_instagram_query = """
INSERT INTO Instagram 
    (account_name, influencer_id, ad_revenue, joining_date, followers, engagement_rate, audience_demographics)
VALUES 
    ('shrishti_instyle', 1, 5000.00, '2020-01-01', 100000, 43.50, '{"age": {"18-24": 30, "25-34": 50, "35-44": 20}, "gender": {"male": 40, "female": 60}}');
"""
execute_query(connection, insert_instagram_query)

insert_posts_query = """
INSERT INTO posts 
    (post_id, account_name, post_date, likes, comments, post_type)
VALUES 
    (1, 'shrishti_instyle', '2022-01-15', 500, 50, 'Image');
"""
execute_query(connection, insert_posts_query)

insert_collaborations_query = """
INSERT INTO collaborations 
    (collaboration_id, brand_name, influencer_id, collaboration_earnings, collaboration_type)
VALUES 
    (1, 'XYZ Brand', 1, 10000.00, 'Product Placement');
"""
execute_query(connection, insert_collaborations_query)

insert_community_engagement_query = """
INSERT INTO community_engagement 
    (engagement_id, influencer_id, event_name, event_date, participants, rating)
VALUES 
    (1, 1, 'Fashion Show', '2022-02-01', 200, 85.5);
"""
execute_query(connection, insert_community_engagement_query)

insert_analytics_query = """
INSERT INTO analytics
VALUES 
    (1, NULL, NULL, NULL, NULL);
"""
execute_query(connection, insert_analytics_query)

Error executing query: 1062 (23000): Duplicate entry '1' for key 'influencers.PRIMARY'
Error executing query: 1062 (23000): Duplicate entry '1-Fashion Design' for key 'course.PRIMARY'
Error executing query: 1062 (23000): Duplicate entry 'shrishti_instyle' for key 'instagram.PRIMARY'
Error executing query: 1062 (23000): Duplicate entry '1' for key 'posts.PRIMARY'
Error executing query: 1062 (23000): Duplicate entry '1' for key 'collaborations.PRIMARY'
Error executing query: 1062 (23000): Duplicate entry '1' for key 'community_engagement.PRIMARY'
Error executing query: 1062 (23000): Duplicate entry '1' for key 'analytics.PRIMARY'


In [ ]:
import pandas as pd
from tabulate import tabulate

from colorama import Fore, Style

def display_menu():
    print("--------", Fore.BLUE + "Influencer Management System" + Style.RESET_ALL, "--------")
    print(f"1. {Fore.GREEN}Show Analytics Table for an Influencer{Style.RESET_ALL}")
    print(f"2. {Fore.YELLOW}Update Course Name{Style.RESET_ALL}")
    print(f"3. {Fore.CYAN}Display Influencers by Category{Style.RESET_ALL}")
    print(f"4. {Fore.MAGENTA}Delete Record from Posts Table{Style.RESET_ALL}")
    print(f"5. {Fore.WHITE}Display All Records in Table Form{Style.RESET_ALL}")
    print(f"6. {Fore.CYAN}New User - Form Filling{Style.RESET_ALL}")
    print(f"7. {Fore.MAGENTA}Add New Record for Collaborations{Style.RESET_ALL}")
    print(f"8. {Fore.WHITE}Show Number of Posts{Style.RESET_ALL}")
    print(f"9. {Fore.YELLOW}Influencers With Image Posts{Style.RESET_ALL}")
    print(f"0. {Fore.RED}Exit{Style.RESET_ALL}")

def show_analytics_table(connection):
    influencer_id = input("Enter Influencer ID: ")
    query = f"SELECT * FROM Analytics WHERE influencer_id = {influencer_id};"
    column_names, data = fetch_data(connection, query)

    if data:
        df = pd.DataFrame(data, columns=column_names)
        print(tabulate(df, headers='keys', tablefmt='fancy_grid'))
    else:
        print("No data found for the given Influencer ID.")
    print("---------------------------------------------------------------------")


def delete_record_from_posts(connection):
    post_id = input("Enter Post ID to Delete: ")
    query = f"DELETE FROM posts WHERE post_id = {post_id};"
    execute_query(connection, query)
    print("Record deleted successfully.")

def update_course_name(connection):
    influencer_id = input("Enter Influencer ID: ")
    course_name = input("Enter New Course Name: ")
    query = f"UPDATE Course SET Course = '{course_name}' WHERE influencer_id = {influencer_id};"
    execute_query(connection, query)
    print("Course name updated successfully.")
    print(" ")
    print("---------------------------------------------------------------------")
    

def display_influencers_by_category(connection):
    category = input("Enter Category (Fashion/Fitness): ")
    query = f"SELECT * FROM influencers WHERE category = '{category}';"
    data = fetch_data(connection, query)
    # Placeholder: Display the influencers using pandas DataFrame
    print(tabulate(data, tablefmt='fancy_grid'))
    print(" ")
    print("---------------------------------------------------------------------")

def display_influencers_by_category(connection):
    category = input("Enter Category (Fashion/Fitness): ")
    query = f"SELECT * FROM influencers WHERE category = '{category}';"
    column_names, data = fetch_data(connection, query)

    if data:
        df = pd.DataFrame(data, columns=column_names)
        print(tabulate(df, headers='keys', tablefmt='fancy_grid'))
    else:
        print(f"No influencers found in the category: {category}.")
    print("---------------------------------------------------------------------")

def display_all_records(connection):
    table_choice = input("Enter Table Name (Influencers/Course/Instagram/Posts/Collaborations/Community_Engagement/Analytics): ")
    query = f"SELECT * FROM {table_choice};"
    column_names, data = fetch_data(connection, query)

    if data:
        df = pd.DataFrame(data, columns=column_names)
        print(tabulate(df, headers='keys', tablefmt='fancy_grid'))
    else:
        print(f"No records found in the table: {table_choice}.")
    print("---------------------------------------------------------------------")

# New function to add a new user with form filling
def new_user_form(connection):
    influencer_id = int(input("Enter id (numeric only): "))
    # Input data for all the columns
    first_name = input("Enter First Name: ")
    last_name = input("Enter Last Name: ")
    dob = input("Enter Date of Birth (YYYY-MM-DD): ")
    gender = input("Enter Gender (M/F): ")
    email = input("Enter Email: ")
    phone_number = input("Enter Phone Number: ")
    date_joined = input("Enter Date Joined (YYYY-MM-DD): ")
    category = input("Enter Category: ")
    assets = float(input("Enter Assets: "))
    liabilities = float(input("Enter Liabilities: "))
    bus_vent_gp = float(input("Enter Business Venture Gross Profit: "))
    expenses = float(input("Enter Expenses: "))

    # SQL query to insert a new user into the influencers table
    query = f"""
    INSERT INTO influencers 
    (influencer_id, first_name, last_name, dob, gender, email, phone_number, date_joined, category, assets, liabilities, bus_vent_gp, expenses)
    VALUES
    ('{influencer_id}','{first_name}', '{last_name}', '{dob}', '{gender}', '{email}', '{phone_number}', '{date_joined}', '{category}',
    {assets}, {liabilities}, {bus_vent_gp}, {expenses});
    """

    # Execute the query to add a new user
    execute_query(connection, query)
    print("New user added successfully.")
    print("---------------------------------------------------------------------")

# New function to add a new record for collaborations
def add_new_collaboration(connection):
    # Input data for collaboration
    collaboration_id = input("Enter collaboration_id: ")
    brand_name = input("Enter Brand Name: ")
    influencer_id = int(input("Enter Influencer ID: "))
    collaboration_earnings = float(input("Enter Collaboration Earnings: "))
    collaboration_type = input("Enter Collaboration Type (Product Placement/Sponsored Event): ")

    # SQL query to insert a new collaboration record into the collaborations table
    query = f"""
    INSERT INTO collaborations 
    (collaboration_id, brand_name, influencer_id, collaboration_earnings, collaboration_type)
    VALUES
    ('{collaboration_id}','{brand_name}', {influencer_id}, {collaboration_earnings}, '{collaboration_type}');
    """

    # Execute the query to add a new collaboration record
    execute_query(connection, query)
    print("New collaboration record added successfully.")

def show_number_of_posts(connection):
    influencer_id = input("Enter Influencer ID: ")
    query = f"SELECT COUNT(*) FROM posts WHERE account_name IN (SELECT account_name FROM Instagram WHERE influencer_id = {influencer_id});"
    num_posts = fetch_data(connection, query)

    if num_posts is not None:
        print(f"Influencer with ID {influencer_id} has made {num_posts} {'post' if num_posts == 1 else 'posts'}.")
    else:
        print(f"No data found for the given Influencer ID.")

def influencers_with_image_posts(connection):
    query = """
    SELECT *
    FROM influencers
    WHERE influencer_id IN (
        SELECT influencer_id
        FROM posts
        WHERE post_type = 'Image'
    );
    """
    column_names, data = fetch_data(connection, query)

    if data:
        df = pd.DataFrame(data, columns=column_names)
        print(tabulate(df, headers='keys', tablefmt='fancy_grid'))
    else:
        print("No influencers found with Image posts.")        

def main():
    your_database_connection = create_connection()

    while True:
        display_menu()
        choice = input("Enter your choice (0-9): ")

        if choice == '1':
            show_analytics_table(connection)
        elif choice == '2':
            update_course_name(connection)
        elif choice == '3':
            display_influencers_by_category(connection)
        elif choice == '4':
            delete_record_from_posts(connection)
        elif choice == '5':
            display_all_records(connection)
        elif choice == '6':
            new_user_form(connection)
        elif choice == '7':
            add_new_collaboration(connection)
        elif choice == '8':
            show_number_of_posts(connection)
        elif choice == '9':
            influencers_with_image_posts(connection)
        elif choice == '0':
            print(f"{Fore.MAGENTA}Exiting Influencer Management System. Goodbye!{Style.RESET_ALL}")        
            break
        else:
            print("Invalid choice. Please enter a number between 0 and 7.")

if __name__ == "__main__":
    main()

Connected to MySQL database
-------- Influencer Management System --------
1. Show Analytics Table for an Influencer
2. Update Course Name
3. Display Influencers by Category
4. Delete Record from Posts Table
5. Display All Records in Table Form
6. New User - Form Filling
7. Add New Record for Collaborations
8. Show Number of Posts
9. Influencers With Image Posts
0. Exit


Enter your choice (0-9):  5
Enter Table Name (Influencers/Course/Instagram/Posts/Collaborations/Community_Engagement/Analytics):  Posts


╒════╤═══════════╤════════════════════════╤═════════════╤═════════╤════════════╤═════════════╕
│    │   post_id │ account_name           │ post_date   │   likes │   comments │ post_type   │
╞════╪═══════════╪════════════════════════╪═════════════╪═════════╪════════════╪═════════════╡
│  0 │         1 │ shrishti_instyle       │ 2022-01-15  │     500 │         50 │ Image       │
├────┼───────────┼────────────────────────┼─────────────┼─────────┼────────────┼─────────────┤
│  1 │         2 │ shrishti_instyle       │ 2022-02-20  │     600 │         60 │ Video       │
├────┼───────────┼────────────────────────┼─────────────┼─────────┼────────────┼─────────────┤
│  2 │         3 │ shrishti_instyle       │ 2022-03-25  │     700 │         70 │ Image       │
├────┼───────────┼────────────────────────┼─────────────┼─────────┼────────────┼─────────────┤
│  3 │         4 │ john_fitness_guru      │ 2022-01-10  │     800 │         80 │ Image       │
├────┼───────────┼────────────────────────┼───────

Enter your choice (0-9):  9


╒════╤═════════════════╤══════════════╤═════════════╤════════════╤══════════╤══════════════════════╤════════════════╤═══════════════╤════════════╤══════════╤═══════════════╤═══════════════╤════════════╕
│    │   influencer_id │ first_name   │ last_name   │ dob        │ gender   │ email                │   phone_number │ date_joined   │ category   │   assets │   liabilities │   bus_vent_gp │   expenses │
╞════╪═════════════════╪══════════════╪═════════════╪════════════╪══════════╪══════════════════════╪════════════════╪═══════════════╪════════════╪══════════╪═══════════════╪═══════════════╪════════════╡
│  0 │               1 │ Shrishti     │ Gupta       │ 1990-05-15 │ F        │ shrishti@example.com │    +1234567890 │ 2020-01-01    │ Fashion    │    50000 │         20000 │         15000 │      10000 │
├────┼─────────────────┼──────────────┼─────────────┼────────────┼──────────┼──────────────────────┼────────────────┼───────────────┼────────────┼──────────┼───────────────┼───────────────